# Model

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, precision_score, recall_score, confusion_matrix, roc_auc_score
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier

import warnings
warnings.filterwarnings("ignore")

<p>Импортируем из БД таблицу работников</p>

In [2]:
import mysql.connector
con = mysql.connector.connect(host='localhost', user='root', password='Pahomov2000+', db='nutritionaldb')

employees = pd.read_sql("SELECT * FROM employees", con)

In [3]:
data = pd.read_csv('E:\\Data Science\\Datasets\\EatingData\\encoded_data_0710.csv')

In [4]:
data

,EmpId,Date,Dish_1_1,Dish_1_2,Dish_1_3,Dish_1_4,Dish_1_5,Dish_1_6,Dish_1_7,Dish_1_8,...,Dish_6_1,Dish_6_2,Dish_6_3,Dish_6_4,Dish_6_5,Dish_6_6,Dish_6_7,Dish_6_8,Dish_6_9,Selected
0,9,2019-03-01,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
1,9,2019-03-01,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
2,9,2019-03-01,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
3,17,2019-03-01,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
4,17,2019-03-01,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239122,418,2020-09-18,0,0,0,0,0,0,0,0,...,0,0,1,1,0,1,1,0,1,1
239123,418,2020-09-18,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0
239124,420,2020-09-18,0,0,0,0,0,0,0,0,...,0,1,0,1,0,1,0,1,1,1
239125,420,2020-09-18,0,0,0,0,0,0,0,0,...,0,0,1,1,0,1,1,0,1,0


<p>Выделяем из таблицы <b>Employees</b> пол сотрудника. Пол определяем по окончанию отчества: если "а" - женский пол, иначе мужской.</p>

In [5]:
sex = []
for emp in data.EmpId:
    if employees[employees['Id'] == emp].Employee.values[0][-1] == 'а':
        sex.append(0)
    else:
        sex.append(1)

data['Sex'] = sex
data

,EmpId,Date,Dish_1_1,Dish_1_2,Dish_1_3,Dish_1_4,Dish_1_5,Dish_1_6,Dish_1_7,Dish_1_8,...,Dish_6_2,Dish_6_3,Dish_6_4,Dish_6_5,Dish_6_6,Dish_6_7,Dish_6_8,Dish_6_9,Selected,Sex
0,9,2019-03-01,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,0
1,9,2019-03-01,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
2,9,2019-03-01,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,0
3,17,2019-03-01,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,0
4,17,2019-03-01,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239122,418,2020-09-18,0,0,0,0,0,0,0,0,...,0,1,1,0,1,1,0,1,1,0
239123,418,2020-09-18,0,0,0,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,0
239124,420,2020-09-18,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,1,1,1
239125,420,2020-09-18,0,0,0,0,0,0,0,0,...,0,1,1,0,1,1,0,1,0,1


<p>Удаляем из данных критерий Дата - поскольку он является неинформативным.</p>
<p>Выделяем столбец <b>Selected</b> - как метку (<b>Label</b>), и конвертируем данные в <i>numpy.arrray</i></p>

In [7]:
del data['Date']

In [8]:
data

,EmpId,Dish_1_1,Dish_1_2,Dish_1_3,Dish_1_4,Dish_1_5,Dish_1_6,Dish_1_7,Dish_1_8,Dish_1_9,...,Dish_6_2,Dish_6_3,Dish_6_4,Dish_6_5,Dish_6_6,Dish_6_7,Dish_6_8,Dish_6_9,Selected,Sex
0,9,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
2,9,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,0
3,17,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,0
4,17,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239122,418,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,1,0,1,1,0
239123,418,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,0
239124,420,0,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,1,1,1
239125,420,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,1,0,1,0,1


In [9]:
label = data.pop('Selected')
label = np.array(label)
data = np.array(data)

In [10]:
# Разделяем данные на тренировочные и тестовые

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.33, random_state=73, shuffle=True)

In [11]:
# Тренируем XGBoost классификатор

model = xgb.XGBClassifier(max_depth=11,
                          subsample=1,
                          n_estimators=400,
                          learning_rate=0.4,
                          reg_alpha=1,
                          reg_lambda=1)

model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.4, max_delta_step=0, max_depth=11,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=400, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=1,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [12]:
# Качество модели

preds = model.predict_proba(X_test)
pr = [np.argmax(i) for i in preds]

print("Accuracy: %.2f%%" % (accuracy_score(y_test, pr) * 100.0))
print("Precision: %.2f%%" % (precision_score(y_test, pr) * 100.0))
print("Recall: %.2f%%" % (recall_score(y_test, pr) * 100.0))
print("ROC-AUC: %.2f%%" % (roc_auc_score(y_test, pr) * 100.0))
print("F1: %.2f%%" % (f1_score(y_test, pr) * 100.0))

Accuracy: 72.75%
Precision: 61.07%
Recall: 51.98%
ROC-AUC: 67.62%
F1: 56.16%
